In [1]:
import random
import time, datetime
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.column as col
import os
from functools import reduce
from pyspark.sql import DataFrame
import subprocess
from datetime import datetime, timedelta
from pyspark.sql.window import Window
from pyspark.sql.types import FloatType, StructType, StructField, LongType
from functools import reduce
import pandas as pd
import pathlib

In [2]:
from pyspark.sql.functions import from_json
from pyspark.sql.types import *

In [4]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.driver.memory=16G --conf spark.driver.maxResultSize=16G pyspark-shell'

spark = (SparkSession
         .builder
         .master("local[*]")
         .getOrCreate())

Exception: Java gateway process exited before sending its port number

## load transacion data

In [ ]:
path = 's3a://midgar-aws-workspace/home/pengcheng/cltv-test/sales_table'

In [ ]:
df = spark.read.parquet(path)


## category

In [ ]:
df = df.select(df.customer_id, df.selling_price.cast(IntegerType()).alias('price'), df.category_id, df.dt).coalesce(1)

In [ ]:
df.show(1)

In [17]:
df.count()

3170420532

In [18]:
cnames = df.select('category_id').distinct()

In [19]:
cnames.count()

2427

In [20]:
catdf = df.groupBy('category_id').agg(
                        F.count('dt').alias('cnt')
                    ).orderBy('cnt', ascending=False)

In [21]:
catdf.show()


'''
+-----------+----------+
|category_id|       cnt|
+-----------+----------+
|     205716|1264392615|
|         17| 818519392|
|         26| 148497864|
|      98945| 129012258|
|         18| 113533144|
|      82653|  60069597|
|     122254|  59955181|
|         21|  48966158|
|      80491|  41515962|
|     126747|  37183492|
|      77409|  33211511|
|     113926|  29905722|
|     100123|  26851209|
|      37217|  22875444|
|     106022|  21725858|
|      25173|  19627366|
|     156705|  18977034|
|     200604|  15378781|
|      69089|  13222374|
|      64739|  13131309|
+-----------+----------+
'''

+-----------+----------+
|category_id|       cnt|
+-----------+----------+
|     205716|1264392615|
|         17| 818519392|
|         26| 148497864|
|      98945| 129012258|
|         18| 113533144|
|      82653|  60069597|
|     122254|  59955181|
|         21|  48966158|
|      80491|  41515962|
|     126747|  37183492|
|      77409|  33211511|
|     113926|  29905722|
|     100123|  26851209|
|      37217|  22875444|
|     106022|  21725858|
|      25173|  19627366|
|     156705|  18977034|
|     200604|  15378781|
|      69089|  13222374|
|      64739|  13131309|
+-----------+----------+
only showing top 20 rows



'\n+-----------+---------+\n|category_id|      cnt|\n+-----------+---------+\n|         17|760750995|\n|         26|135873546|\n|         18|113533144|\n|      98945| 87996115|\n|      82653| 60069597|\n|     122254| 59950949|\n|         21| 44045340|\n|      80491| 40574247|\n|      77409| 33137547|\n|     126747| 31837532|\n|     113926| 29905722|\n|     100123| 26851209|\n|      37217| 20225230|\n|      25173| 18603704|\n|     106022| 17982724|\n|     156705| 17174837|\n|     200604| 15167349|\n|      69089| 13185369|\n|      64739| 12986611|\n|      99932| 10580326|\n+-----------+---------+\n'

### category data

In [6]:
traing_start_date = '2019-01-01'
traing_end_date = '2019-06-30'
testing_start_date = '2019-07-01'
testing_end_date = '2019-12-31'
date_format_str ='yyyy-MM-dd'

In [7]:
print(traing_start_date, traing_end_date)

2019-01-01 2019-06-30


In [8]:
print(testing_start_date, testing_end_date)

2019-07-01 2019-12-31


In [9]:
def history_by_category(df, start_date, end_date, category_id):
        
    df_agg = df.where((df.dt >= start_date) & (df.dt <= end_date) & (df.category_id == category_id)).groupBy('customer_id')\
                    .agg(
                        F.countDistinct('dt').alias('tx_cnt'),
                        F.avg('price').cast(FloatType()).alias('monetary_value'),
                        F.sum('price').cast(FloatType()).alias('total_value'),
                        F.max('dt').alias('max_dt'),
                        F.min('dt').alias('min_dt')
                    ).withColumn('last_date', F.lit(end_date)).withColumn('first_date', F.lit(start_date))
        
    df_agg = df_agg.withColumn('frequency', F.col('tx_cnt')-1)\
                   .withColumn('T', 
                               F.datediff(
                                   F.to_date('last_date', date_format_str),
                                   F.to_date('min_dt', date_format_str)))\
                   .withColumn('recency',
                               F.when(df_agg.tx_cnt == 1, 0.0).
                               otherwise(F.datediff(
                                   F.to_date('max_dt', date_format_str),
                                   F.to_date('min_dt', date_format_str)
                               ))
                              )
 
    
    return df_agg.select(df_agg.customer_id, df_agg.frequency, df_agg.recency, df_agg.monetary_value, df_agg.T, df_agg.total_value)
    #return df_agg

In [10]:
def history_by_category_agg(df, start_date, end_date):
        
    df_agg = df.where((df.dt >= start_date) & (df.dt <= end_date)).groupBy('customer_id', 'category_id')\
                    .agg(
                        F.countDistinct('dt').alias('tx_cnt'),
                        F.avg('price').cast(FloatType()).alias('monetary_value'),
                        F.max('dt').alias('max_dt'),
                        F.min('dt').alias('min_dt')
                    ).withColumn('last_date', F.lit(end_date)).withColumn('first_date', F.lit(start_date))
        
    df_agg = df_agg.withColumn('frequency', F.col('tx_cnt')-1)\
                   .withColumn('T', 
                               F.datediff(
                                   F.to_date('last_date', date_format_str),
                                   F.to_date('min_dt', date_format_str)))\
                   .withColumn('recency',
                               F.when(df_agg.tx_cnt == 1, 0.0).
                               otherwise(F.datediff(
                                   F.to_date('max_dt', date_format_str),
                                   F.to_date('min_dt', date_format_str)
                               ))
                              )
 
    
    return df_agg.select(df_agg.customer_id, df.category_id, df_agg.frequency, df_agg.recency, df_agg.monetary_value, df_agg.T)

In [11]:
def gen_train_test_data(df, traing_start_date, traing_end_date, 
                        testing_start_date, testing_end_date, 
                        cid, sample_ratio=1.0):
    
    df_train = history_by_category(df, traing_start_date, traing_end_date, cid)
    df_test = history_by_category(df, testing_start_date, testing_end_date, cid)
    
    train_path = '/tmp/data/pengcheng/cltv-summary/' + str(cid) + ".pkl"
    test_path = '/tmp/data/pengcheng/cltv-summary/' + str(cid) + "_testing.pkl"
    
    if sample_ratio == 1.0:
        if os.path.exists(train_path):
            os.remove(train_path)
        df_train.toPandas().to_pickle(train_path)
        
        if os.path.exists(test_path):
            os.remove(test_path)
        df_test.toPandas().to_pickle(test_path)
        
        gen_customer_category_date(df, df_train, cid)
        
    else:
        part_train, _ = df_train.randomSplit([sample_ratio, 1-sample_ratio])
        if os.path.exists(train_path):
            os.remove(train_path)
        part_train.toPandas().to_pickle(train_path)
        
        part_test = part_train.select(part_train.customer_id).join(df_test, on=['customer_id'])
        if os.path.exists(test_path):
            os.remove(test_path)
        part_test.toPandas().to_pickle(test_path)
        
        gen_customer_category_date(df, part_train, cid)
    

In [12]:
def gen_customer_category_date(df, df_summary, cid):
    
    id1 = df_summary.where(df_summary.frequency == 2).select('customer_id').collect()[0].customer_id
    id2 = df_summary.where(df_summary.frequency == 10).select('customer_id').collect()[0].customer_id

    txns = df.where(df.customer_id.isin([id1, id2]))
    folder = '/tmp/data/pengcheng/cltv-customer/'
    
    if not os.path.exists(folder):
        os.mkdir(folder)
        
    path = folder + str(cid) + '.csv'
    txns.coalesce(1).write.mode('overwrite').format("com.databricks.spark.csv").option("header", "true").save(path)
    

In [30]:
df = df.select(df.customer_id, 
               df.selling_price.cast(IntegerType())
               .alias('price'), 
               df.category_id, 
               df.dt).filter(df.category_id == 26)


In [31]:
gen_train_test_data(df, traing_start_date, traing_end_date, 
                        testing_start_date, testing_end_date, 26, sample_ratio=0.3)

In [42]:
#gen_customer_category_date(df, [87935,1135220295], 17)

In [31]:
#gen_customer_category_date(df, 3708702, 17)

In [74]:
cst=df.where(df.customer_id.isin([15432,13452]))

In [75]:
cst.select('customer_id').distinct().head(4)

[Row(customer_id=15432), Row(customer_id=13452)]

In [76]:
(cst.coalesce(1).write.mode('overwrite')
.format("com.databricks.spark.csv")
.option("header", "true")
.save('/tmp/data/pengcheng/cltv-customer/26.csv'))

In [51]:
ddd = pd.DataFrame(list(range(5)), columns=['d'])

In [52]:
ddd.describe()

,d
count,5.000000
mean,2.000000
std,1.581139
min,0.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,4.000000


In [54]:
ddd['d'].rank(pct=True)

0    0.2
1    0.4
2    0.6
3    0.8
4    1.0
Name: d, dtype: float64